# 02a Prep regional statistic data 

> A first look at the ONS data on a parliamentary constituency level. This includes total population, age, ethnicity and educational attainment.   

---

In [1]:
#|default_exp core.02_prep_regional_data

In [2]:
#|hide
import nbdev; nbdev.nbdev_export()

In [3]:
#|hide
from nbdev.showdoc import show_doc

In [4]:
#|export
import dementia_inequalities as proj
from dementia_inequalities import const, log, utils, tools
import adu_proj.utils as adutils

In [5]:
#|export
import numpy as np 
import pandas as pd 

/var/folders/4n/x6w1yfcx01qbymrsfpz4ybq00000gn/T/ipykernel_1663/3424933337.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


---

## Prep parliamentary constituency data 

Start by loading the datasets: 
* mid-2020 parliamentary constituency population estimates by age
* 2021 census ethnicity data
* 2021 census educational qualifications data. 

In [6]:
#|export
df_pc_age = pd.read_excel(const.data_path+'/PC_data/mid2020parliconsyoaestimatesage.xlsx', sheet_name='Mid-2020 Persons')
df_pc_eth = pd.read_excel(const.data_path+'/PC_data/ethnicity_2021census.xlsx')
df_pc_educ = pd.read_excel(const.data_path+'/PC_data/qualifications_2021census.xlsx')

/Users/bea/miniconda3/envs/adu_env_dementia_ineq/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [7]:
# Tidy table 
df_pc_age.columns = df_pc_age.iloc[3]
df_pc_age.drop(df_pc_age.index[0:4], inplace=True)

# Add columns for >65 age 
age_65_loc = df_pc_age.columns.get_loc(65.0)
df_pc_age['over_65'] = df_pc_age.iloc[:,age_65_loc:].sum(axis=1)
df_pc_age['over_65_pc'] = df_pc_age['over_65']/df_pc_age['All Ages']

In [8]:
df_pc_age.head()

3,PCON11CD,PCON11NM,All Ages,0.0,1.0,2.0,3.0,4.0,5.0,6.0,...,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90+,over_65,over_65_pc
4,E14000530,Aldershot,105168,1313.0,1401.0,1436.0,1294.0,1347.0,1491.0,1323.0,...,449.0,362.0,317.0,322.0,230.0,186.0,179.0,802,16472.0,0.156626
5,E14000531,Aldridge-Brownhills,77683,783.0,789.0,840.0,784.0,822.0,908.0,897.0,...,568.0,461.0,412.0,348.0,333.0,319.0,253.0,922,18114.0,0.233178
6,E14000532,Altrincham and Sale West,102444,943.0,1058.0,1130.0,1198.0,1390.0,1287.0,1416.0,...,511.0,455.0,436.0,376.0,346.0,311.0,292.0,1252,19395.0,0.189323
7,E14000533,Amber Valley,92277,815.0,902.0,932.0,1008.0,957.0,964.0,939.0,...,466.0,409.0,350.0,327.0,318.0,237.0,191.0,897,20266.0,0.219621
8,E14000534,Arundel and South Downs,102673,789.0,779.0,903.0,938.0,984.0,1097.0,1052.0,...,737.0,690.0,590.0,552.0,496.0,436.0,427.0,1562,29079.0,0.28322


In [9]:
# pivot from long format to wide for education and ethnicity datasets 
df_pc_educ_wide = df_pc_educ[['RegNationName','ConstituencyName', 'groups', 'Con_pc']].pivot_table(index=['RegNationName','ConstituencyName'], columns='groups', values='Con_pc')
df_pc_eth_wide = df_pc_eth[['ConstituencyName', 'ethnic_groups', 'Con_pc']].pivot_table(index='ConstituencyName', columns='ethnic_groups', values='Con_pc')

In [10]:
df_pc_educ_wide.head()

groups                             1 or more GCSEs  2 or more A levels  \
RegNationName ConstituencyName                                           
East Midlands Amber Valley                0.131008            0.183863   
              Ashfield                    0.139701            0.176188   
              Bassetlaw                   0.129686            0.173597   
              Bolsover                    0.133019            0.177657   
              Boston and Skegness         0.136154            0.145172   

groups                             Apprenticeship  \
RegNationName ConstituencyName                      
East Midlands Amber Valley               0.070313   
              Ashfield                   0.068853   
              Bassetlaw                  0.067468   
              Bolsover                   0.067650   
              Boston and Skegness        0.071770   

groups                             Higher education qualifications  \
RegNationName ConstituencyName                                       
East Midlands Amber Valley                                0.243244   
              Ashfield                                    0.205517   
              Bassetlaw                                   0.251465   
              Bolsover                                    0.222011   
              Boston and Skegness                         0.182568   

groups                             No qualifications  Other qualifications  
RegNationName ConstituencyName                                              
East Midlands Amber Valley                  0.216873              0.023691  
              Ashfield                      0.244210              0.025830  
              Bassetlaw                     0.217066              0.031033  
              Bolsover                      0.239934              0.026709  
              Boston and Skegness           0.286914              0.041268

In [11]:
df_pc_eth_wide.head()

ethnic_groups,African,Any other ethnic group,Arab,Bangladeshi,Caribbean,Chinese,"English, Welsh, Scottish, Northern Irish or British",Gypsy or Irish Traveller,Indian,Irish,Other Asian,Other Black,Other Mixed or Multiple ethnic groups,Other White,Pakistani,Roma,White and Asian,White and Black African,White and Black Caribbean
ConstituencyName,,,,,,,,,,,,,,,,,,,
Aberavon,0.002926,0.003042,0.000750,0.005795,0.001024,0.003431,0.935000,0.001989,0.005781,0.003373,0.005709,0.000793,0.002725,0.014646,0.002465,0.000346,0.004022,0.001658,0.004527
Aberconwy,0.001253,0.002345,0.000895,0.001253,0.000609,0.003168,0.938102,0.000555,0.002452,0.007715,0.003258,0.000519,0.002757,0.026259,0.000859,0.000233,0.003938,0.001539,0.002291
Aldershot,0.014307,0.024408,0.001770,0.002867,0.006237,0.005024,0.722020,0.001905,0.020625,0.006659,0.099161,0.004080,0.006471,0.052896,0.011359,0.001024,0.008295,0.004098,0.006794
Aldridge-Brownhills,0.007418,0.009324,0.000451,0.001172,0.009156,0.003503,0.864846,0.000721,0.043000,0.006619,0.004636,0.002267,0.003941,0.014243,0.009130,0.000193,0.006568,0.001365,0.011449
Altrincham and Sale West,0.010701,0.013912,0.010959,0.002398,0.004033,0.027615,0.751504,0.000109,0.044579,0.018806,0.009641,0.001784,0.008075,0.034581,0.036582,0.000614,0.011454,0.003934,0.008720


Let's check the length of each dataframe. They should include all the English parliamentary constituencies (533) plus all the Welsh constituencies (40). 

In [12]:
print(len(df_pc_eth_wide))
print(len(df_pc_educ_wide))
print(len(df_pc_age))

573
573
573


---

## Prep local authority districts data 

Start by loading the datasets: 
* mid-2021 local auhtority population estimates by age
* 2021 census ethnicity data
* 2021 census educational qualifications data. 

> **NOTE** It's important to use the population estimates from 2021, as the local authority districts changed in 2023 - so the latest population estimates are for different areas. 

In [13]:
#|export
df_lad_age = pd.read_excel(const.data_path+'/LAD_data/mid_21_age_pop.xlsx', sheet_name='Population')
df_lad_eth = pd.read_excel(const.data_path+'/LAD_data/ethnicity_2021_census.xlsx')
df_lad_educ = pd.read_excel(const.data_path+'/LAD_data/education_2021_census.xlsx', sheet_name='Dataset')

In [14]:
# Just the data from 2021
df_lad_age_21 = df_lad_age[df_lad_age['time']==2021].copy()
df_lad_age_21.columns = ['LAD_code', 'LAD_name', 'Year', 'Sex', 'Age', 'Population_mean', 'credible_int_lower', 'credible_int_upper']

# pivot table 
df_lad_age_wide = df_lad_age_21[['LAD_name', 'Sex', 'Age', 'Population_mean']].pivot_table(index=['LAD_name', 'Sex'], columns='Age', values='Population_mean')
df_lad_age_wide.reset_index(inplace=True)

In [15]:
# add columns for age > 65
age_65_lad_loc = df_lad_age_wide.columns.get_loc(65)
age_0_lad_loc = df_lad_age_wide.columns.get_loc(0)
age_90_lad_loc = df_lad_age_wide.columns.get_loc(90)

df_lad_age_wide['all_age'] = df_lad_age_wide.iloc[:,age_0_lad_loc:age_90_lad_loc+1].sum(axis=1)
df_lad_age_wide['over_65'] = df_lad_age_wide.iloc[:,age_65_lad_loc:age_90_lad_loc+1].sum(axis=1)

In [16]:
df_lad_age_wide.head()

Age,LAD_name,Sex,0,1,2,3,4,5,6,7,...,83,84,85,86,87,88,89,90,all_age,over_65
0,Adur,female,270.0,298.9934,347.9799,324.5530,350.8981,351.7227,384.3616,364.1633,...,270.8853,218.7755,201.9398,192.9233,154.0302,142.9736,126.9776,582.9755,33367.2174,8582.3745
1,Adur,male,284.0,288.0070,319.0459,333.2678,351.6067,386.4282,377.0479,401.8020,...,172.0647,176.7854,143.8455,140.8996,101.6105,92.7852,87.9722,263.0277,31234.9942,6710.0072
2,Allerdale,female,409.0,410.0282,401.9808,431.0913,464.2950,449.0778,488.0521,510.2826,...,358.9272,310.9886,283.9464,262.0119,211.9530,211.9523,195.9545,769.0074,49249.0560,12811.5675
3,Allerdale,male,428.0,398.9811,427.9839,478.9639,462.2162,476.1545,478.4490,465.3898,...,270.9098,275.9271,219.8636,182.9583,153.9684,133.9809,128.9914,345.3928,47134.0962,11219.2449
4,Amber Valley,female,547.0,547.0101,600.0238,631.0501,610.9781,631.1320,632.1657,640.4586,...,371.9216,384.9295,317.9503,265.0028,262.9864,238.0400,174.9937,816.0006,64358.8403,15258.6783


In [17]:
# pivot again! for a nicer format

# pivot the data across all ages
df_lad_age_wide_all = df_lad_age_wide[['LAD_name', 'Sex', 'all_age']].pivot_table(index='LAD_name', columns='Sex', values='all_age')
df_lad_age_wide_all['all_age'] = df_lad_age_wide_all['female']+df_lad_age_wide_all['male']
df_lad_age_wide_all.reset_index(inplace=True)
df_lad_age_wide_all.columns = ['LAD_name', 'Female_all', 'Male_all', 'all_age']

# pivot the data across all ages over 65
df_lad_age_wide_65 = df_lad_age_wide[['LAD_name', 'Sex', 'over_65']].pivot_table(index='LAD_name', columns='Sex', values='over_65')
df_lad_age_wide_65['over_65'] = df_lad_age_wide_65['female']+df_lad_age_wide_65['male']
df_lad_age_wide_65.reset_index(inplace=True)
df_lad_age_wide_65.columns = ['LAD_name', 'Female_over_65', 'Male_over_65', 'over_65']

In [18]:
# finally merge them into a df that only contains the useful info
df_lad_age_21 = pd.merge(df_lad_age_wide_all, df_lad_age_wide_65, on='LAD_name')

In [19]:
df_lad_age_21.head()

,LAD_name,Female_all,Male_all,all_age,Female_over_65,Male_over_65,over_65
0,Adur,33367.2174,31234.9942,64602.2116,8582.3745,6710.0072,15292.3817
1,Allerdale,49249.0560,47134.0962,96383.1522,12811.5675,11219.2449,24030.8124
2,Amber Valley,64358.8403,62133.1758,126492.0161,15258.6783,13404.6739,28663.3522
3,Arun,85623.0905,79503.6255,165126.7160,25993.5877,21146.1918,47139.7795
4,Ashfield,64559.0530,61844.1711,126403.2241,13493.5435,11521.6532,25015.1967


In [20]:
# tidy the ethnicity dataset
df_lad_eth.columns = df_lad_eth.iloc[3]
df_lad_eth.drop(df_lad_eth.index[0:4], inplace=True)
df_lad_eth.reset_index(inplace=True)
df_lad_eth.drop(axis=1, labels='index', inplace=True)

In [21]:
df_lad_eth.head()

3,Area code,Area name,"Asian, Asian British or Asian Welsh: Bangladeshi\n(number)","Asian, Asian British or Asian Welsh: Chinese\n(number)","Asian, Asian British or Asian Welsh: Indian\n(number)","Asian, Asian British or Asian Welsh: Pakistani\n(number)","Asian, Asian British or Asian Welsh: Other Asian\n(number)","Black, Black British, Black Welsh, Caribbean or African: African\n(number)","Black, Black British, Black Welsh, Caribbean or African: Caribbean\n(number)","Black, Black British, Black Welsh, Caribbean or African: Other Black\n(number)",...,Mixed or Multiple ethnic groups: White and Black African \n(percent),Mixed or Multiple ethnic groups: White and Black Caribbean\n(percent),Mixed or Multiple ethnic groups: Other Mixed or Multiple ethnic groups\n(percent),"White: English, Welsh, Scottish, Northern Irish or British\n(percent)",White: Irish\n(percent),White: Gypsy or Irish Traveller\n(percent),White: Roma\n(percent),White: Other White\n(percent),Other ethnic group: Arab\n(percent),Other ethnic group: Any other ethnic group\n(percent)
0,E06000001,Hartlepool,278,217,335,297,473,327,57,61,...,0.1,0.2,0.2,95,0.2,0,0,1.2,0.3,0.3
1,E06000002,Middlesbrough,595,669,2804,8990,2032,3339,162,315,...,0.5,0.4,0.5,79.5,0.3,0.1,0.2,2.2,1,1.4
2,E06000003,Redcar and Cleveland,158,208,175,283,336,182,48,35,...,0.2,0.2,0.2,96.5,0.3,0.1,0,0.8,0.2,0.2
3,E06000004,Stockton-on-Tees,236,690,1812,4875,1439,1823,130,250,...,0.2,0.2,0.4,90.3,0.3,0.1,0.1,1.3,0.3,0.6
4,E06000005,Darlington,759,308,1086,195,618,456,135,110,...,0.2,0.4,0.3,90.3,0.3,0.4,0.1,3.3,0.3,0.6


In [22]:
# Seperate the dataset with percentages 
x = list(df_lad_eth.filter(regex='percent').columns)

x.insert(0, 'Area name')
x.insert(0, 'Area code')

df_lad_eth_percent = df_lad_eth[x]

In [23]:
df_lad_eth_percent.head()

3,Area code,Area name,"Asian, Asian British or Asian Welsh: Bangladeshi\n(percent)","Asian, Asian British or Asian Welsh: Chinese\n(percent)","Asian, Asian British or Asian Welsh: Indian\n(percent)","Asian, Asian British or Asian Welsh: Pakistani\n(percent)","Asian, Asian British or Asian Welsh: Other Asian\n(percent)","Black, Black British, Black Welsh, Caribbean or African: African\n(percent)","Black, Black British, Black Welsh, Caribbean or African: Caribbean\n(percent)","Black, Black British, Black Welsh, Caribbean or African: Other Black\n(percent)",...,Mixed or Multiple ethnic groups: White and Black African \n(percent),Mixed or Multiple ethnic groups: White and Black Caribbean\n(percent),Mixed or Multiple ethnic groups: Other Mixed or Multiple ethnic groups\n(percent),"White: English, Welsh, Scottish, Northern Irish or British\n(percent)",White: Irish\n(percent),White: Gypsy or Irish Traveller\n(percent),White: Roma\n(percent),White: Other White\n(percent),Other ethnic group: Arab\n(percent),Other ethnic group: Any other ethnic group\n(percent)
0,E06000001,Hartlepool,0.3,0.2,0.4,0.3,0.5,0.4,0.1,0.1,...,0.1,0.2,0.2,95,0.2,0,0,1.2,0.3,0.3
1,E06000002,Middlesbrough,0.4,0.5,1.9,6.2,1.4,2.3,0.1,0.2,...,0.5,0.4,0.5,79.5,0.3,0.1,0.2,2.2,1,1.4
2,E06000003,Redcar and Cleveland,0.1,0.2,0.1,0.2,0.2,0.1,0,0,...,0.2,0.2,0.2,96.5,0.3,0.1,0,0.8,0.2,0.2
3,E06000004,Stockton-on-Tees,0.1,0.4,0.9,2.5,0.7,0.9,0.1,0.1,...,0.2,0.2,0.4,90.3,0.3,0.1,0.1,1.3,0.3,0.6
4,E06000005,Darlington,0.7,0.3,1,0.2,0.6,0.4,0.1,0.1,...,0.2,0.4,0.3,90.3,0.3,0.4,0.1,3.3,0.3,0.6


In [24]:
# pivot the education dataset
df_lad_educ_wide = df_lad_educ[['Lower tier local authorities', 'Highest level of qualification (8 categories)', 'Observation']].pivot_table(index=
                                                                                                                          'Lower tier local authorities', columns='Highest level of qualification (8 categories)', values='Observation')
df_lad_educ_wide.reset_index(inplace=True)
df_lad_educ_wide.columns = ['Area name', 'Apprenticeship', 'not applicable', 'level 1: 4 GCSEs', 'level 2: 5 GCSEs', 'level 3: a-level', 'level 4: bsc', 'no qualifications', 'other']

In [25]:
df_lad_educ_wide.head()

,Area name,Apprenticeship,not applicable,level 1: 4 GCSEs,level 2: 5 GCSEs,level 3: a-level,level 4: bsc,no qualifications,other
0,Adur,3325.0,11591.0,6065.0,7949.0,9055.0,14784.0,10333.0,1440.0
1,Allerdale,6020.0,15641.0,7969.0,11162.0,13887.0,22855.0,16609.0,2011.0
2,Amber Valley,7032.0,21143.0,10975.0,15009.0,18723.0,30265.0,20668.0,2388.0
3,Arun,9101.0,25335.0,16142.0,21323.0,23983.0,36839.0,27534.0,4630.0
4,Ashfield,6949.0,23454.0,12299.0,16138.0,18393.0,22185.0,24281.0,2602.0


In [26]:
len(df_lad_educ_wide)

331

In [27]:
# Need to decide which .csv files to save 

df_lad_age_21.to_csv(const.output_path+'/LAD_age_2021.csv', index=False)
df_lad_eth_percent.to_csv(const.output_path+'/LAD_ethnicity_percent_2021.csv', index=False)
df_lad_educ_wide.to_csv(const.output_path+'/LAD_education_2021.csv', index=False)